# Feature engineering

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

import xgboost

import sklearn
from sklearn.model_selection import train_test_split

import sys, os, gc, types
import time
from subprocess import check_output

In [37]:
root_paths = [
    "/data/kaggle-instacart/",
    "/Users/jiayou/Dropbox/珺珺的程序/Kaggle/Instacart/",
    "/Users/jiayou/Dropbox/Documents/珺珺的程序/Kaggle/Instacart/"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(check_output(["ls", root]).decode("utf8"))

Build Feature.ipynb
CHANGELOG.md
F1 score optimization.ipynb
Instacart Data Exploration.ipynb
Param Search Results.ipynb
README.md
Tasks.ipynb
Toy DF.ipynb
Training.ipynb
aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
sample_submission.csv
submission-v0.csv
submission-v1-r0.csv
submission-v1-r1.csv
training-logs
utils



In [38]:
def load_data(path_data):
    priors = pd.read_csv(path_data + 'order_products__prior.csv', 
                     dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    train = pd.read_csv(path_data + 'order_products__train.csv', 
                    dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    orders = pd.read_csv(path_data + 'orders.csv', 
                         dtype={
                                'order_id': np.int32,
                                'user_id': np.int64,
                                'eval_set': 'category',
                                'order_number': np.int16,
                                'order_dow': np.int8,
                                'order_hour_of_day': np.int8,
                                'days_since_prior_order': np.float32})
    
    products = pd.read_csv(path_data + 'products.csv')
    aisles = pd.read_csv(path_data + "aisles.csv")
    departments = pd.read_csv(path_data + "departments.csv")
    sample_submission = pd.read_csv(path_data + "sample_submission.csv")
    
    return priors, train, orders, products, aisles, departments, sample_submission

# orders_df and order_products__prior/train_df shares the 'order_id'
# order_products__prior/train_df and products_df shares the 'product_id'
# products_df and aisles_df shares the 'aisles_id'
# products_df and departments_df shares the 'department id'

In [39]:
class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))
            
def ka_add_groupby_features_1_vs_n(df, group_columns_list, agg_dict, only_new_feature=True):
    with tick_tock("add stats features"):
        try:
            if type(group_columns_list) == list:
                pass
            else:
                raise TypeError(k + "should be a list")
        except TypeError as e:
            print(e)
            raise

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped.agg(agg_dict)
        the_stats.columns = the_stats.columns.droplevel(0)
        the_stats.reset_index(inplace=True)
        if only_new_feature:
            df_new = the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')

    return df_new

def ka_add_groupby_features_n_vs_1(df, group_columns_list, target_columns_list, methods_list, keep_only_stats=True, verbose=1):
    with tick_tock("add stats features", verbose):
        dicts = {"group_columns_list": group_columns_list , "target_columns_list": target_columns_list, "methods_list" :methods_list}

        for k, v in dicts.items():
            try:
                if type(v) == list:
                    pass
                else:
                    raise TypeError(k + "should be a list")
            except TypeError as e:
                print(e)
                raise

        grouped_name = ''.join(group_columns_list)
        target_name = ''.join(target_columns_list)
        combine_name = [[grouped_name] + [method_name] + [target_name] for method_name in methods_list]

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped[target_name].agg(methods_list).reset_index()
        the_stats.columns = [grouped_name] + \
                            ['_%s_%s_by_%s' % (grouped_name, method_name, target_name) \
                             for (grouped_name, method_name, target_name) in combine_name]
        if keep_only_stats:
            return the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')
        return df_new


In [49]:
priors, train, orders, products, aisles, departments, sample_submission = load_data(root)

In [51]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered,user_id
0,2,33120,1,1,202279
1,2,28985,2,1,202279
2,2,9327,3,0,202279
3,2,45918,4,1,202279
4,2,30035,5,0,202279


In [52]:
# down_sample = None
down_sample = 10
if down_sample is not None:
    priors = priors.merge(orders[['order_id', 'user_id']], on='order_id', how='left')
    train = train.merge(orders[['order_id', 'user_id']], on='order_id', how='left')
    
    orders = orders[orders.user_id % down_sample == 0]
    priors = priors[priors.user_id % down_sample == 0]
    train = train[train.user_id % down_sample == 0]
    
    priors.drop('user_id', inplace = True, axis=1)
    train.drop('user_id', inplace = True, axis=1)

In [53]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered
9,3,33754,1,1
10,3,24838,2,1
11,3,17704,3,1
12,3,21903,4,1
13,3,17668,5,1


# Product feature engineering

In [54]:
# build days_to_last_order
dsfo = orders.groupby('user_id').days_since_prior_order.cumsum().fillna(0)
orders['days_since_first_order'] = dsfo
max_days = orders.groupby('user_id').days_since_first_order.agg({'max_days':'max'}).reset_index()
orders = orders.merge(max_days, on = 'user_id', how = 'left')
orders['days_to_last_order'] = orders.max_days - orders.days_since_first_order
orders['hod_group'] = (orders.order_hour_of_day / 4).astype('int')

In [55]:
orders.drop(['days_since_first_order', 'max_days'], axis=1, inplace=True)
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,days_to_last_order,hod_group
0,1224907,10,prior,1,2,14,NaN,109.0,3
1,68288,10,prior,2,5,15,30.0,79.0,3
2,2115522,10,prior,3,3,19,12.0,67.0,4
3,83395,10,prior,4,3,15,14.0,53.0,3
4,1353310,10,prior,5,5,20,23.0,30.0,5


In [56]:
priors_orders_detail = orders.merge(right=priors, how='inner', on='order_id').merge(products[['product_id','aisle_id']], how = 'left', on = 'product_id')
priors_orders_detail.loc[:,'user_buy_product_times'] = priors_orders_detail.groupby(['user_id', 'product_id']).cumcount() + 1
priors_orders_detail.loc[:,'user_buy_category_times'] = priors_orders_detail.groupby(['user_id', 'aisle_id']).cumcount() + 1
agg_dict = {'user_id':{'prod_total_cnt':'count'}, 
            'reordered':{'prod_reorder_total_cnt':'sum'}, 
            'user_buy_product_times': {'prod_user_cnt':lambda x: sum(x==1),
                                        'prod_return_user_cnt':lambda x: sum(x==2)}}
prd = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['product_id'], agg_dict)

prd['prod_user_reorder_ratio'] = prd.prod_return_user_cnt / prd.prod_user_cnt
prd['prod_product_reorder_ratio'] = prd.prod_reorder_total_cnt / prd.prod_total_cnt

add stats features begin ......
add stats features end ......
time lapsing 47.2529411315918 s 



In [57]:
prd.head()

,product_id,prod_total_cnt,prod_reorder_total_cnt,prod_user_cnt,prod_return_user_cnt,prod_user_reorder_ratio,prod_product_reorder_ratio
0,1,214,138.0,76,33,0.434211,0.644860
1,2,13,3.0,10,3,0.300000,0.230769
2,3,6,2.0,4,2,0.500000,0.333333
3,4,38,20.0,18,4,0.222222,0.526316
4,5,7,6.0,1,1,1.000000,0.857143


In [58]:
priors_orders_detail.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,days_to_last_order,hod_group,product_id,add_to_cart_order,reordered,aisle_id,user_buy_product_times,user_buy_category_times
0,1224907,10,prior,1,2,14,NaN,109.0,3,46979,1,0,83,1,1
1,1224907,10,prior,1,2,14,NaN,109.0,3,24852,2,0,24,1,1
2,1224907,10,prior,1,2,14,NaN,109.0,3,27104,3,0,83,1,2
3,1224907,10,prior,1,2,14,NaN,109.0,3,16797,4,0,24,1,2
4,1224907,10,prior,1,2,14,NaN,109.0,3,31717,5,0,16,1,1


# Product-Timeline feature engineering

In [59]:
agg_dict_8 = {'order_id':{'prod_cnt_by_hod':'count'}}
prod_hod = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['product_id', 'hod_group'], agg_dict_8)
prod_timeline = prod_hod.merge(prd[['product_id', 'prod_total_cnt']], on = 'product_id', how = 'left')
prod_timeline['prod_market_share_hod'] = prod_timeline['prod_cnt_by_hod'] / prod_timeline['prod_total_cnt']

agg_dict_9 = {'order_id':{'prod_cnt_by_dow':'count'}}
prod_dow = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['product_id', 'order_dow'], agg_dict_9)
prod_timeline_2 = prod_dow.merge(prd[['product_id', 'prod_total_cnt']], on = 'product_id', how = 'left')
prod_timeline_2['prod_market_share_dow'] = prod_timeline_2['prod_cnt_by_dow'] / prod_timeline_2['prod_total_cnt']

add stats features begin ......
add stats features end ......
time lapsing 0.6526310443878174 s 

add stats features begin ......
add stats features end ......
time lapsing 0.6933069229125977 s 



In [60]:
prod_timeline_2.head(7)

,product_id,order_dow,prod_cnt_by_dow,prod_total_cnt,prod_market_share_dow
0,1,0,33,214,0.154206
1,1,1,56,214,0.261682
2,1,2,26,214,0.121495
3,1,3,25,214,0.116822
4,1,4,24,214,0.112150
5,1,5,33,214,0.154206
6,1,6,17,214,0.079439


# Category feature engineering

In [61]:
agg_dict_5 = {'user_id':{'cat_total_bought_cnt':'count'}, 
            'reordered':{'cat_reorder_total_cnt':'sum'}, 
            'user_buy_category_times': {'cat_user_cnt':lambda x: sum(x==1),
                                        'cat_return_user_cnt':lambda x: sum(x==2)}}
cat = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['aisle_id'], agg_dict_5)

cat['cat_user_reorder_ratio'] = cat.cat_return_user_cnt / cat.cat_user_cnt
cat['cat_product_reorder_ratio'] = cat.cat_reorder_total_cnt / cat.cat_total_bought_cnt

agg_dict_6 = {'user_buy_category_times':{'cat_user_bought_cnts':'max'}}
cat_agg = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['aisle_id', 'user_id'], agg_dict_6)

agg_dict_7 = {'cat_user_bought_cnts':{'cat_num_of_prods_a_user_buys_in_this_cat_mean':'mean',
                                     'cat_num_of_prods_a_user_buys_in_this_cat_std':'std',
                                     'cat_num_of_prods_a_user_buys_in_this_cat_max':'max',
                                     'cat_num_of_prods_a_user_buys_in_this_cat_median':'median'}}
category_agg = ka_add_groupby_features_1_vs_n(cat_agg, ['aisle_id'], agg_dict_7)
category = cat.merge(category_agg, on = 'aisle_id', how = 'left')
category.head()

add stats features begin ......
add stats features end ......
time lapsing 29.09045386314392 s 

add stats features begin ......
add stats features end ......
time lapsing 0.819605827331543 s 

add stats features begin ......
add stats features end ......
time lapsing 0.06858515739440918 s 



,aisle_id,cat_total_bought_cnt,cat_reorder_total_cnt,cat_user_cnt,cat_return_user_cnt,cat_user_reorder_ratio,cat_product_reorder_ratio,cat_num_of_prods_a_user_buys_in_this_cat_mean,cat_num_of_prods_a_user_buys_in_this_cat_std,cat_num_of_prods_a_user_buys_in_this_cat_max,cat_num_of_prods_a_user_buys_in_this_cat_median
0,1,7258,4403.0,2047,1071,0.523205,0.606641,3.545677,5.935688,78,2
1,2,8506,4320.0,3116,1492,0.478819,0.507877,2.729782,4.180075,68,1
2,3,46552,27885.0,6391,4677,0.731810,0.599008,7.283993,15.021891,428,3
3,4,20147,9907.0,5451,3400,0.623739,0.491736,3.696019,4.519952,60,2
4,5,6434,1875.0,3305,1255,0.379728,0.291421,1.946747,2.177548,45,1


# User feature engineering

In [62]:
agg_dict_2 = {'order_number':{'user_total_orders':'max'},
              'days_since_prior_order':{'user_sum_days_since_prior_order':'sum', 
                                        'user_mean_days_since_prior_order': 'mean'}}
users = ka_add_groupby_features_1_vs_n(orders[orders.eval_set == 'prior'], ['user_id'], agg_dict_2)

agg_dict_3 = {'reordered':
              {'user_reorder_ratio': 
               lambda x: sum(priors_orders_detail.ix[x.index,'reordered']==1)/
                         sum(priors_orders_detail.ix[x.index,'order_number'] > 1)},
              'product_id':{'user_total_products':'count', 
                            'user_distinct_products':'nunique'}}
us = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['user_id'], agg_dict_3)
users = users.merge(us, how='inner')

add stats features begin ......
add stats features end ......
time lapsing 0.044818878173828125 s 

add stats features begin ......
add stats features end ......
time lapsing 79.31802606582642 s 



In [63]:
users['user_average_basket'] = users.user_total_products / users.user_total_orders

us = orders[orders.eval_set != "prior"]
# us.rename(index=str, columns={'days_since_prior_order': 'time_since_last_order'}, inplace=True)

users = users.merge(us, how='inner')

In [64]:
us.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,days_to_last_order,hod_group
5,1822501,10,train,6,0,19,30.0,0.0,4
10,1980631,20,test,5,1,11,30.0,0.0,2
19,62370,30,train,9,2,13,22.0,0.0,3
29,2431024,40,test,10,0,8,7.0,0.0,2
97,1750084,50,train,68,3,9,7.0,0.0,2


In [65]:
users.head()

,user_id,user_total_orders,user_sum_days_since_prior_order,user_mean_days_since_prior_order,user_reorder_ratio,user_total_products,user_distinct_products,user_average_basket,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,days_to_last_order,hod_group
0,10,5,79.0,19.750000,0.355072,143,94,28.600000,1822501,train,6,0,19,30.0,0.0,4
1,20,4,15.0,5.000000,0.833333,22,7,5.500000,1980631,test,5,1,11,30.0,0.0,2
2,30,8,151.0,21.571428,0.625000,11,6,1.375000,62370,train,9,2,13,22.0,0.0,3
3,40,9,105.0,13.125000,0.739130,104,36,11.555556,2431024,test,10,0,8,7.0,0.0,2
4,50,67,357.0,5.409091,0.814318,453,89,6.761194,1750084,train,68,3,9,7.0,0.0,2


# User - Product feature engineering

In [66]:
agg_dict_4 = {'order_number':{'up_order_count': 'count', 
                              'up_first_order_number': 'min', 
                              'up_last_order_number':'max'}, 
              'add_to_cart_order':{'up_average_cart_position': 'mean'},
              'days_to_last_order':{'up_days_since_last_order':'min'}}

data = ka_add_groupby_features_1_vs_n(df=priors_orders_detail, 
                                                      group_columns_list=['user_id', 'product_id', 'aisle_id'], 
                                                      agg_dict=agg_dict_4)

data = data.merge(prd, how='inner', on='product_id').merge(users, how='inner', on='user_id').merge(category, how = 'inner', on='aisle_id')

data['up_order_rate'] = data.up_order_count / data.user_total_orders
data['up_order_since_last_order'] = data.user_total_orders - data.up_last_order_number
data['up_order_rate_since_first_order'] = data.up_order_count / (data.user_total_orders - data.up_first_order_number + 1)

train = train.merge(right=orders[['order_id', 'user_id']], how='left', on='order_id')
data = data.merge(train[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')
data = data.merge(
    prod_timeline[['product_id', 'hod_group', 'prod_market_share_hod']], 
    on = ['product_id', 'hod_group'], 
    how = 'left')
data.prod_market_share_hod.fillna(0)
data = data.merge(
    prod_timeline_2[['product_id', 'order_dow', 'prod_market_share_dow']], 
    on = ['product_id', 'order_dow'], how = 'left')
data.prod_market_share_dow.fillna(0)

add stats features begin ......
add stats features end ......
time lapsing 1.5060288906097412 s 



0          0.196550
1          0.206415
2          0.189266
3          0.192834
4          0.202289
5          0.200943
6          0.207511
7          0.190333
8          0.217149
9          0.138150
10         0.175291
11         0.178804
12         0.185876
13         0.165169
14         0.174592
15         0.217999
16         0.170459
17         0.173996
18         0.181539
19         0.170886
20         0.147597
21         0.174356
22         0.145594
23         0.196308
24         0.185123
25         0.170672
26         0.181921
27         0.177561
28         0.128761
29         0.125084
             ...   
1338227    0.222222
1338228    0.045455
1338229    0.000000
1338230    0.230769
1338231    0.235294
1338232    0.110390
1338233    0.220779
1338234    0.220779
1338235    0.000000
1338236    0.133333
1338237    0.166667
1338238    0.000000
1338239    0.038462
1338240    0.088235
1338241    0.181818
1338242    0.250000
1338243    0.166667
1338244    0.129870
1338245    0.000000


In [68]:
data.shape

(1338257, 45)

In [69]:
del priors_orders_detail, orders
gc.collect()

884

In [70]:
drop_list = ['user_id', 'aisle_id', 'order_number', 'order_dow', 'order_hour_of_day', 'days_to_last_order', 'hod_group', 'cat_num_of_prods_a_user_buys_in_this_cat_median']
data.drop(drop_list, inplace = True, axis=1)


In [71]:
data.shape

(1338257, 37)

In [72]:
for col in data.columns:
    if data[col].dtypes == 'float64':
        data[col] = data[col].astype('float32')
    if data[col].dtypes == 'int64':
        data[col] = data[col].astype('int32')
data.dtypes

product_id                                         int32
up_order_count                                     int32
up_first_order_number                              int16
up_last_order_number                               int16
up_average_cart_position                         float32
up_days_since_last_order                         float32
prod_total_cnt                                     int32
prod_reorder_total_cnt                           float32
prod_user_cnt                                      int32
prod_return_user_cnt                               int32
prod_user_reorder_ratio                          float32
prod_product_reorder_ratio                       float32
user_total_orders                                  int16
user_sum_days_since_prior_order                  float32
user_mean_days_since_prior_order                 float32
user_reorder_ratio                               float32
user_total_products                                int32
user_distinct_products         

In [73]:
data_train = data[data.eval_set == 'train']
data_test = data[data.eval_set == 'test']

In [74]:
# data.to_csv(os.path.join(root, 'abt.csv'), index = None)
data_train.to_csv(os.path.join(root, 'abt_train.csv'), index = None)
data_test.to_csv(os.path.join(root, 'abt_test.csv'), index = None)